Various imports

In [1]:
import numpy as np

from keras.layers import Dense, Input, Flatten, Reshape, concatenate, Dropout
from keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Embedding
from keras.layers import LSTM, Bidirectional
from keras.models import Model
from keras import optimizers
from keras import regularizers
import random
from sklearn.utils import shuffle
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
from gensim.models import word2vec

Using TensorFlow backend.


Read in Movie Review Dataset

In [0]:
maxlen = 32
num_words = 88585
embed = 32
filter_size = 32

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words,seed=42)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
vocabulary = imdb.get_word_index()
vocabulary_inv = dict((v, k) for k, v in vocabulary.items())
vocabulary_inv[0] = "<PAD/>"

Create the CNN-rand model

In [0]:
sequence_input = Input(shape=(maxlen,), dtype='int32')

e = Embedding(num_words, embed, input_length=maxlen)(sequence_input)

e = Reshape((maxlen, embed, 1))(e)

x = Conv2D(filter_size, (5, embed), activation='relu')(e)
x = MaxPooling2D((3, 1))(x)
x = Flatten()(x)

y = Conv2D(filter_size, (4, embed), activation='relu')(e)
y = MaxPooling2D((3, 1))(y)
y = Flatten()(y)

z = Conv2D(filter_size, (3, embed), activation='relu')(e)
z = MaxPooling2D((3, 1))(z)
z = Flatten()(z)

alpha = concatenate([x,y,z])

alpha = Dropout(0.5)(alpha)

layer_a = Dense(128, activation='relu')(alpha)

layer_b = Dense(64, activation='relu')(layer_a)

layer_c = Dense(32, activation='relu')(layer_b)

preds = Dense(1, activation='sigmoid')(layer_c)

model = Model(sequence_input, preds)
    
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

Train the model and print accuracy

In [33]:
model.fit(x_train, y_train,
          batch_size=256,
          epochs=5,
          validation_data=(x_test, y_test))
print(" ")
print("---Results---")
print(" ")
print("Training accuracy:")
print(model.evaluate(x_train, y_train)[1])
print(" ")
print("Testing accuracy:")
print(model.evaluate(x_test, y_test)[1])

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 65s 3ms/step - loss: 0.4065 - acc: 0.7975 - val_loss: 0.2829 - val_acc: 0.8814
Epoch 2/5
25000/25000 [==============================] - 59s 2ms/step - loss: 0.1300 - acc: 0.9526 - val_loss: 0.3181 - val_acc: 0.8746
Epoch 3/5
25000/25000 [==============================] - 59s 2ms/step - loss: 0.0368 - acc: 0.9889 - val_loss: 0.4681 - val_acc: 0.8643
Epoch 4/5
25000/25000 [==============================] - 59s 2ms/step - loss: 0.0110 - acc: 0.9969 - val_loss: 0.9060 - val_acc: 0.8413
Epoch 5/5
25000/25000 [==============================] - 59s 2ms/step - loss: 0.0049 - acc: 0.9981 - val_loss: 0.8148 - val_acc: 0.8652
 
---Results---
 
Training accuracy:
25000/25000 [==============================] - 18s 731us/step
0.99928
 
Testing accuracy:
25000/25000 [==============================] - 18s 726us/step
0.86524


For CNN-non-static and CNN-static embedding weights must be calculated using the word2vec module

In [15]:
sentences = [[vocabulary_inv[w] for w in s] for s in np.vstack((x_train, x_test))]
embedding_model = word2vec.Word2Vec(sentences,
                                    size=embed, 
                                    min_count=1,
                                    window=10, 
                                    sample=1e-3)
embedding_weights = {key: embedding_model[word] if word in embedding_model else
                              np.random.uniform(-0.25, 0.25, embedding_model.vector_size)
                         for key, word in vocabulary_inv.items()}

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


For the CNN-non-static the only modification that has to be made is adding the embedding weights to the Embedding layer

In [0]:
sequence_input = Input(shape=(maxlen,), dtype='int32')

e = Embedding(num_words, embed, input_length=maxlen, name="embedding")(sequence_input)

e = Reshape((maxlen, embed, 1))(e)

x = Conv2D(filter_size, (5, embed), activation='relu')(e)
x = MaxPooling2D((3, 1))(x)
x = Flatten()(x)

y = Conv2D(filter_size, (4, embed), activation='relu')(e)
y = MaxPooling2D((3, 1))(y)
y = Flatten()(y)

z = Conv2D(filter_size, (3, embed), activation='relu')(e)
z = MaxPooling2D((3, 1))(z)
z = Flatten()(z)

alpha = concatenate([x,y,z])

alpha = Dropout(0.5)(alpha)

layer_a = Dense(128, activation='relu')(alpha)

layer_b = Dense(64, activation='relu')(layer_a)

layer_c = Dense(32, activation='relu')(layer_b)

preds = Dense(1, activation='sigmoid')(layer_c)

model = Model(sequence_input, preds)
    
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

Add the weights to the embeddnig layer, train the model and print accuracy

In [10]:
weights = np.array([v for v in embedding_weights.values()])
embedding_layer = model.get_layer("embedding")
embedding_layer.set_weights([weights])

model.fit(x_train, y_train,
          batch_size=256,
          epochs=5,
          validation_data=(x_test, y_test))

print(" ")
print("---Results---")
print(" ")
print("Training accuracy:")
print(model.evaluate(x_train, y_train)[1])
print(" ")
print("Testing accuracy:")
print(model.evaluate(x_test, y_test)[1])

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 11s 451us/step - loss: 0.6260 - acc: 0.6235 - val_loss: 0.4238 - val_acc: 0.8067
Epoch 2/5
25000/25000 [==============================] - 10s 393us/step - loss: 0.3025 - acc: 0.8739 - val_loss: 0.3322 - val_acc: 0.8561
Epoch 3/5
25000/25000 [==============================] - 10s 393us/step - loss: 0.1373 - acc: 0.9496 - val_loss: 0.3841 - val_acc: 0.8509
Epoch 4/5
25000/25000 [==============================] - 10s 393us/step - loss: 0.0491 - acc: 0.9848 - val_loss: 0.5084 - val_acc: 0.8420
Epoch 5/5
25000/25000 [==============================] - 10s 393us/step - loss: 0.0179 - acc: 0.9949 - val_loss: 0.7183 - val_acc: 0.8380
 
---Results---
 
Training accuracy:
25000/25000 [==============================] - 4s 154us/step
0.99804
 
Testing accuracy:
25000/25000 [==============================] - 4s 152us/step
0.83796


For the CNN-static we must first modify the x_train and y_test vectors

In [0]:
x_train = np.stack([np.stack([embedding_weights[word] for word in sentence]) for sentence in x_train])
x_test = np.stack([np.stack([embedding_weights[word] for word in sentence]) for sentence in x_test])

And modify the model to accept the new input shape. In edition we have no need for the embedding layer since the embeddings are already found

In [17]:
sequence_input = Input(shape=(maxlen,embed), dtype='float32')

e = Reshape((maxlen, embed, 1))(sequence_input)

x = Conv2D(filter_size, (5, embed), activation='relu')(e)
x = MaxPooling2D((3, 1))(x)
x = Flatten()(x)

y = Conv2D(filter_size, (4, embed), activation='relu')(e)
y = MaxPooling2D((3, 1))(y)
y = Flatten()(y)

z = Conv2D(filter_size, (3, embed), activation='relu')(e)
z = MaxPooling2D((3, 1))(z)
z = Flatten()(z)

alpha = concatenate([x,y,z])

alpha = Dropout(0.5)(alpha)

layer_a = Dense(128, activation='relu')(alpha)

layer_b = Dense(64, activation='relu')(layer_a)

layer_c = Dense(32, activation='relu')(layer_b)

preds = Dense(1, activation='sigmoid')(layer_c)

model = Model(sequence_input, preds)
    
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Train and print accuracy

In [18]:
model.fit(x_train, y_train,
          batch_size=256,
          epochs=5,
          validation_data=(x_test, y_test))
print(" ")
print("---Results---")
print(" ")
print("Training accuracy:")
print(model.evaluate(x_train, y_train)[1])
print(" ")
print("Testing accuracy:")
print(model.evaluate(x_test, y_test)[1])

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 2s 64us/step - loss: 0.6238 - acc: 0.6337 - val_loss: 0.5462 - val_acc: 0.7176
Epoch 2/5
25000/25000 [==============================] - 1s 32us/step - loss: 0.5388 - acc: 0.7238 - val_loss: 0.5199 - val_acc: 0.7321
Epoch 3/5
25000/25000 [==============================] - 1s 32us/step - loss: 0.5151 - acc: 0.7389 - val_loss: 0.5206 - val_acc: 0.7322
Epoch 4/5
25000/25000 [==============================] - 1s 31us/step - loss: 0.5029 - acc: 0.7481 - val_loss: 0.5121 - val_acc: 0.7377
Epoch 5/5
25000/25000 [==============================] - 1s 31us/step - loss: 0.4870 - acc: 0.7571 - val_loss: 0.4929 - val_acc: 0.7506
 
---Results---
 
Training accuracy:
25000/25000 [==============================] - 1s 57us/step
0.79104
 
Testing accuracy:
25000/25000 [==============================] - 1s 56us/step
0.75056
